In [128]:
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import dill as pickle
from IPython.core.display import HTML
import helpers.data as data_helper
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [129]:
reload(data_helper)
Y_true = data_helper.get_labeled_data()
Y_true.head()

,document_id,type,sentence_index,start_word_index,end_word_index,string
entity_id,,,,,,
bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on:5:5,bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on,ORDINAL,0,5,5,first
bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on:200:201,bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on,PERSON,18,19,20,Ye Daying
bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on:228:229,bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on,PERSON,18,47,48,Ye Ting
bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on:245:245,bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on,ORDINAL,19,11,11,second
bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on:237:238,bc/phoenix/00/phoenix_0000@0000@phoenix@bc@en@on,PERSON,19,3,4,Ye Zhengming


In [130]:
!ls worker/*.csv

worker/Y_pred.20181207.csv


In [133]:
Y_pred = pd.read_csv('worker/Y_pred.20181207.csv')
Y_pred = Y_pred.set_index('entity_id')
Y_pred.head()

,prediction
entity_id,
wb/sel/94/sel_9442@9442@sel@wb@en@on:2:6,WORK_OF_ART
wb/sel/22/sel_2236@2236@sel@wb@en@on:10:10,FAC
pt/nt/57/nt_5701@5701@nt@pt@en@on:2:2,PERSON
pt/nt/57/nt_5701@5701@nt@pt@en@on:7:8,GPE
pt/nt/57/nt_5701@5701@nt@pt@en@on:12:12,PERSON


In [134]:
documents = data_helper.get_documents()
documents.head()

,document
document_id,
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,"In the summer of 2005 , a picture that people ..."
bc/cctv/00/cctv_0001@0001@cctv@bc@en@on,What kind of memory ? We respectfully invite y...
bc/cctv/00/cctv_0002@0002@cctv@bc@en@on,Abramov had a car accident in Moscow last nigh...
bc/cctv/00/cctv_0003@0003@cctv@bc@en@on,"Hello , dear viewers . Welcome to Focus Today ..."
bc/cctv/00/cctv_0004@0004@cctv@bc@en@on,There will be 120 million viewers tuning in to...


In [135]:
def find_string_in_document(entity_id):
    document_id, start_index, end_index = entity_id.split(':')
    start_index = int(start_index)
    end_index = int(end_index)
    document = documents.loc[document_id]
    words = document.document.split(' ')
    return ' '.join(words[start_index:end_index+1])

Y_pred['string'] = [find_string_in_document(entity_id) for entity_id, _ in Y_pred.iterrows()]

In [136]:
Y_pred.head()

,prediction,string
entity_id,,
wb/sel/94/sel_9442@9442@sel@wb@en@on:2:6,WORK_OF_ART,the United States of America
wb/sel/22/sel_2236@2236@sel@wb@en@on:10:10,FAC,State
pt/nt/57/nt_5701@5701@nt@pt@en@on:2:2,PERSON,Paul
pt/nt/57/nt_5701@5701@nt@pt@en@on:7:8,GPE,Jesus Christ
pt/nt/57/nt_5701@5701@nt@pt@en@on:12:12,PERSON,Timothy


In [137]:
Y_pred.iloc[100]

prediction    WORK_OF_ART
string            a month
Name: wb/sel/51/sel_5132@5132@sel@wb@en@on:12:13, dtype: object

In [138]:
!ls ../data/

dev_document_ids.pkl  download.sh		  sentences.20181205.pkl
doc0_ytrue.csv	      entity_ids.pkl		  test_document_ids.pkl
document.csv	      LP_NET_Graphs.20181205.pkl  train_document_ids.pkl
documents_dev.pkl     LP_NET_Graphs.pkl		  www.glozman.com
documents.pkl	      name_entity.csv		  ypred.csv
documents_test.pkl    NET_Graphs.pkl		  Y_pred.csv
documents_train.pkl   sentence.csv		  Y_true.pkl


In [139]:
with open('../data/dev_document_ids.pkl', 'rb') as file:
    document_ids = pickle.load(file)

document_ids[0]

'wb/sel/23/sel_2368@2368@sel@wb@en@on'

In [140]:
Y_true_dev = Y_true[Y_true.document_id.isin(document_ids)]
len(Y_true_dev)

32580

In [141]:
Y_pred.head()

,prediction,string
entity_id,,
wb/sel/94/sel_9442@9442@sel@wb@en@on:2:6,WORK_OF_ART,the United States of America
wb/sel/22/sel_2236@2236@sel@wb@en@on:10:10,FAC,State
pt/nt/57/nt_5701@5701@nt@pt@en@on:2:2,PERSON,Paul
pt/nt/57/nt_5701@5701@nt@pt@en@on:7:8,GPE,Jesus Christ
pt/nt/57/nt_5701@5701@nt@pt@en@on:12:12,PERSON,Timothy


In [142]:
def build_comparison_matrix(y_true_df, y_pred_df):
    Y_true = []
    Y_pred = []
    
    entity_ids = set(list(y_true_df.index) + list(y_pred_df.index))
    
    for entity_id in tqdm(entity_ids):
        try:
            true_type = y_true_df.loc[entity_id].type
            pred_type = y_pred_df.loc[entity_id].prediction
        except KeyError:
            continue

        Y_true.append(true_type)
        Y_pred.append(pred_type)
    
    return [Y_true, Y_pred]

Y_true_list, Y_pred_list = build_comparison_matrix(Y_true_dev, Y_pred)
display(len(Y_true_list))
display(len(Y_pred_list))

100%|██████████| 53237/53237 [00:09<00:00, 5374.23it/s]


19323

19323

In [144]:
print(classification_report(Y_true_list, Y_pred_list, digits=4))

             precision    recall  f1-score   support

   CARDINAL     0.2352    0.2448    0.2399      1601
       DATE     0.4317    0.2719    0.3337      2883
      EVENT     0.0169    0.0862    0.0283       116
        FAC     0.0411    0.1208    0.0613       149
        GPE     0.3546    0.2390    0.2856      3527
   LANGUAGE     0.0167    0.4286    0.0321         7
        LAW     0.0319    0.0923    0.0474        65
        LOC     0.0364    0.0967    0.0529       300
      MONEY     0.2891    0.4499    0.3521       829
       NORP     0.2114    0.2855    0.2429      1408
   NO_MATCH     0.0000    0.0000    0.0000         0
    ORDINAL     0.0677    0.2222    0.1038       360
        ORG     0.4378    0.1943    0.2691      3891
    PERCENT     0.2041    0.1647    0.1823       668
     PERSON     0.4257    0.3640    0.3924      2937
    PRODUCT     0.0340    0.0977    0.0505       133
   QUANTITY     0.0378    0.0924    0.0536       184
       TIME     0.0525    0.1244    0.0739   

/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [145]:
def per_category_accuracy_scores(y_true_df, y_pred_df):
    print('Accuracy:')
    print('{: <13} {: <8} {: <7}'.format('Category', 'Accuracy', 'Support'))
    categories = ['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 
                  'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 
                  'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']
    for category in categories:
        category_true = y_true_df[y_true_df.type == category]
        Y_true = []
        Y_pred = []
        for entity_id, row in category_true.iterrows():
            try:
                true_type = y_true_df.loc[entity_id].type
                pred_type = y_pred_df.loc[entity_id].prediction
            except KeyError:
                continue

            Y_true.append(true_type)
            Y_pred.append(pred_type)
        print('{: <13} {:8.1f} {:7d}'.format(
            category,
            accuracy_score(Y_true, Y_pred) * 100,
            len(Y_true)
        ))
        
per_category_accuracy_scores(Y_true_dev, Y_pred)

Accuracy:
Category      Accuracy Support
CARDINAL          24.5    1601
DATE              27.2    2883
EVENT              8.6     116
FAC               12.1     149
GPE               23.9    3527
LANGUAGE          42.9       7
LAW                9.2      65
LOC                9.7     300
MONEY             45.0     829
NORP              28.6    1408
ORDINAL           22.2     360
ORG               19.4    3891
PERCENT           16.5     668
PERSON            36.4    2937
PRODUCT            9.8     133
QUANTITY           9.2     184
TIME              12.4     201
WORK_OF_ART       15.6      64


In [146]:
Y_pred.head()

,prediction,string
entity_id,,
wb/sel/94/sel_9442@9442@sel@wb@en@on:2:6,WORK_OF_ART,the United States of America
wb/sel/22/sel_2236@2236@sel@wb@en@on:10:10,FAC,State
pt/nt/57/nt_5701@5701@nt@pt@en@on:2:2,PERSON,Paul
pt/nt/57/nt_5701@5701@nt@pt@en@on:7:8,GPE,Jesus Christ
pt/nt/57/nt_5701@5701@nt@pt@en@on:12:12,PERSON,Timothy


In [147]:
len(Y_pred[Y_pred.prediction == 'NO_MATCH'])

420